In [ ]:

import numpy as np

# =============================================================================
# definition of environment and inputs
# =============================================================================
#the shape of the environment
environment_rows = 5 
environment_columns = 5

start = (0, 0) #start point

#hole1 = (2, 0) #hole1 point
#hole2 = (2, 1) #hole2 point
#wall = (3, 2) #wall point

#goal = (4, 4) #goal point

episodes = 500 #one sequence of states, actions, and rewards, which ends with a terminal state

epsilon = 0.5 #the percentage of time when we should take the best action (instead of a random action)
discount_factor = 0.8 #discount factor for future rewards

# =============================================================================
# Q-Learning class
# =============================================================================

dq_values = {}
class Q_Learning:
    
    def __init__(self, environment_rows, environment_columns, start, hole1, hole2,hole3, hole4,  goal, episodes, epsilon, discount_factor):
        self.environment_rows = environment_rows
        self.environment_columns = environment_columns
        self.start = start
        self.hole1 = hole1
        self.hole2 = hole2
        self.hole3 = hole3
        self.hole4 = hole4
        self.goal = goal
        self.episodes = episodes
        self.epsilon = epsilon
        self.discount_factor = discount_factor
        
    def set_matrices(self):
        #create a 2D numpy array to hold the rewards for each state
        self.rewards = np.full((self.environment_rows, self.environment_columns), -1)
        self.rewards[self.hole1] = self.rewards[self.hole2]= self.rewards[self.hole3]= self.rewards[self.hole4]  = -100
        self.rewards[self.goal] = 100
        
        #create a 3D numpy array to hold the current Q-values for each state and action pair: Q(s, a)
        

        #define actions
        #numeric action codes: 0 = up, 1 = right, 2 = down, 3 = left
        self.actions = ['up', 'right', 'down', 'left']
    
    #define a function that determines if the specified location is a terminal state
    def is_terminal_state(self, current_row_index, current_column_index):
        if self.rewards[current_row_index, current_column_index] == -1:
            return False
        else:
            return True
    def append_state(self,current_row_index, current_column_index):
        if current_row_index-1<0:
          rewup=-100
        else:
          rewup= self.rewards[current_row_index-1,current_column_index]
        if current_column_index+1>=self.environment_columns:
          rewright=-100
        else:
          rewright=self.rewards[current_row_index,current_column_index+1]
        if current_row_index+1>=self.environment_rows:
          rewdown=-100
        else:
          rewdown=self.rewards[current_row_index+1,current_column_index]
        if current_column_index-1<0:
          rewleft=-100
        else:
          rewleft= self.rewards[current_row_index,current_column_index-1]
        
        state=(rewup,rewright,rewdown,rewleft)
        if state not in dq_values:     #if state not present add in table
              dq_values.update({state:[0,0,0,0]})
        return state

    #define an epsilon greedy algorithm that will choose which action to take next
    def get_next_action(self, current_row_index, current_column_index, epsilon):
        #if a randomly chosen value between 0 and 1 is less than epsilon, 
        #then choose the most promising value from the Q-table for this state.
        
        state=self.append_state(current_row_index,current_column_index)
        if np.random.random() < epsilon:   #exploit
            return np.argmax(dq_values[state])
        else:                              #explore
            return np.random.randint(4)

    #define a function that will get the next location based on the chosen action
    def get_next_location(self, current_row_index, current_column_index, action_index):
        new_row_index = current_row_index
        new_column_index = current_column_index
        if self.actions[action_index] == 'up' and current_row_index > 0:
            new_row_index -= 1
        elif self.actions[action_index] == 'right' and current_column_index < self.environment_columns - 1:
            new_column_index += 1
        elif self.actions[action_index] == 'down' and current_row_index < self.environment_rows - 1:
            new_row_index += 1
        elif self.actions[action_index] == 'left' and current_column_index > 0:
            new_column_index -= 1
        return new_row_index, new_column_index


    #run through 500 training episodes
    def train(self):
        for episode in range(self.episodes):
            #get the starting location for this episode
            row_index, column_index = self.start

            #continue taking actions (i.e., moving) until we reach a terminal state
            #(i.e., until we reach the item packaging area or crash into an item storage location)
            while not self.is_terminal_state(row_index, column_index):
      
                #choose which action to take (i.e., where to move next)
                action_index = self.get_next_action(row_index, column_index, self.epsilon)

                #store the old row and column indexes
                old_row_index, old_column_index = row_index, column_index 
    
                #perform the chosen action, and transition to the next state (i.e., move to the next location)
                row_index, column_index = self.get_next_location(row_index, column_index, action_index)
    
                #receive the reward for moving to the new state, and calculate the temporal difference
                reward = self.rewards[row_index, column_index]

                #new_q_value = reward + (self.discount_factor * np.max(self.q_values[row_index, column_index]))
                state=self.append_state(row_index, column_index)
                new_q_value= reward + (self.discount_factor * np.max(dq_values[state]))
                state_prev=self.append_state(old_row_index, old_column_index)

                #update the Q-value for the previous state and action pair
                dq_values[state_prev][action_index] = new_q_value
        print(dq_values)
        print(len(dq_values))
        print("----------------------")   


    def get_shortest_path(self, start):
        start_row_index, start_column_index = start
        #return immediately if this is an invalid starting location
        if self.is_terminal_state(start_row_index, start_column_index):
            return []
        else: #if this is a 'legal' starting location
            current_row_index, current_column_index = start_row_index, start_column_index
            shortest_path = []
            shortest_path.append([current_row_index, current_column_index])
            #continue moving along the path until we reach the goal (i.e., the item packaging location)
            while not self.is_terminal_state(current_row_index, current_column_index):
                #get the best action to take
                action_index = self.get_next_action(current_row_index, current_column_index, 1)
                #move to the next location on the path, and add the new location to the list
                current_row_index, current_column_index = self.get_next_location(current_row_index, current_column_index, action_index)
                print(current_row_index,current_column_index)

                shortest_path.append([current_row_index, current_column_index])
        return shortest_path


    def get_board_path(self,start,shortest_path):
      board=self.rewards
      for moves in shortest_path:
        if board[moves[0]][moves[1]]!=100:
          board[moves[0]][moves[1]]=2
        
      for i in range(environment_rows):
        for j in range(environment_columns):
          if board[i][j]==-1:
            print("_",end=' ')
          if board[i][j]==-100:
            print("X",end=' ')
          if board[i][j]==100:
            print("G",end=' ')
          if board[i][j]==2:
            print("O",end=' ') 
        print("\n") 




In [ ]:
# =============================================================================
#     
# =============================================================================
Q_L = Q_Learning(environment_rows, environment_columns, start, (0,2), (0,3), (1,4),(2,0),(4,4), episodes, epsilon, discount_factor)
Q_L.set_matrices()
Q_L.rewards
Q_L.train()
print("finished training for 1st case")
# shortest_path=Q_L.get_shortest_path(start)
# print(shortest_path)
print("################################################")

Q_L1 = Q_Learning(environment_rows, environment_columns, start, (2,0), (2,1), (3,2),(0,4) ,(4,4), episodes, epsilon, discount_factor)
Q_L1.set_matrices()
Q_L1.rewards
Q_L1.train()
print("finished training for 2nd case")
# shortest_path1=Q_L1.get_shortest_path(start)
# print(shortest_path1)
# print("################################################")

Q_L2 = Q_Learning(environment_rows, environment_columns, start, (2,1), (0,2), (3,0), (4,3),(4,4), episodes, epsilon, discount_factor)
Q_L2.set_matrices()
Q_L2.rewards
Q_L2.train()
print("finished training for 3rd case")
shortest_path2=Q_L2.get_shortest_path(start)

print(shortest_path2)

{(-100, -1, -1, -100): [9.092861440000005, 9.092861440000005, 9.092861440000005, 9.092861440000005], (-100, -100, -1, -1): [-100.0, 4.019431321600004, 6.274289152000005, 4.019431321600004], (-1, -1, -100, -100): [12.616076800000005, 6.274289152000005, -92.72571084799999, 12.616076800000005], (-1, -1, -1, -100): [9.092861440000005, 17.020096000000006, 17.020096000000006, -86.3839232], (-1, -1, -1, -1): [2.2155450572800035, 4.019431321600004, 4.019431321600004, 4.019431321600004], (-100, -1, -1, -1): [-60.99199999999999, 6.274289152000005, 9.092861440000005, 12.616076800000005], (-1, -100, 100, -1): [62.2, 79.0, 100.0, 6.274289152000005], (-1, -100, -100, -1): [0, 0, 0, 0], (-1, -1, -100, -1): [17.020096000000006, 79.0, 48.760000000000005, 48.760000000000005], (-1, -100, -1, -1): [0, 0, 0, 0], (-1, 100, -100, -1): [48.760000000000005, 100.0, 79.0, 62.2]}
11
----------------------
finished training for 1st case
################################################
{(-100, -1, -1, -100): [9.092

In [ ]:
Q_L.get_board_path(start,shortest_path)
print("---------------------")
Q_L1.get_board_path(start,shortest_path1)

O _ X X _ 

O O _ _ X 

_ O _ _ _ 

_ O _ _ _ 

_ O O O G 

---------------------
O _ _ _ _ 

O O O O O 

X X _ _ O 

_ _ X _ O 

_ _ _ _ G 

